# Algoritmo de Kruskal / Kruskal algorithm

Vamos a implementar el algoritmo de Kruskal

<p align="center">
  <img src="imgs/psc_kruskal_tfg.png">
</p>

Aquí vamos a comenzar tratando con **grafos no dirigidos**.

La manera de representar el grafo va a ser basada en la clase ***lisgraph*** mencionada en el libro *Fundamentals of algorithmics. Chapter 5: Some data structures (Brassard, G.; Bratley, P.; 1996)*. De ahí generaremos la lista de enlaces y la ordenaremos por longitud ascendente.

La clase ***lisgraph*** se basa en un diccionario donde:
* Las claves son las etiquetas de los vértices del grafo. Asumimos que son números enteros.
* Los valores son listas de tuplas (etiqueta, longitud) donde se representan al vértice vecino y la longitud del enlace que los conecta.

Por otro lado, los enlaces vamos a representarlos como diccionarios donde:
* Las claves son "e" (proviene de "enlace") y "l" (proviene de "longitud")
* Los valores son un conjunto {etiqueta1, etiqueta2} para el enlace y un número entero para la longitud

In [1]:
# Ejemplo de enlace no dirigido como diccionario
a = { "e" : {"1","2"},
      "l" : 1 }

<p align="center">
  <img src="imgs/grafo_ejemplo_tfg.png">
</p>

In [2]:
# Grafo de ejemplo del TFG, representado como lisgraph
g = { "1" : [("2", 3), ("3", 4), ("5", 4)],
      "2" : [("1", 3), ("4", 5)],
      "3" : [("1", 4), ("4", 1), ("5", 2), ("6", 4)],
      "4" : [("2", 5), ("3", 1), ("5", 3), ("6", 1)],
      "5" : [("1", 4), ("3", 2), ("4", 3)],
      "6" : [("3", 4), ("4", 1)] }

In [3]:
'''
    Clase que representa un grafo ponderado NO DIRIGIDO 
    como un diccionario con:
    - Las etiquetas de los vértices como claves
    - Una lista con tuplas de (etiqueta, longitud) de los vecinos
      de dicho vértice y con cada longitud de esos enlaces
    Se implementan luego algunos métodos útiles
'''

class lisgraph(object):
    
    def __init__(self, grafo_dic=None):
        '''
            Inicializa un objeto lisgraph.
            Si no se pasa el diccionario,
            se usará uno vacío.
        '''
        if grafo_dic == None:
            grafo_dic = {}
        self.__grafo_dic = grafo_dic

    def vertices(self):
        ''' Devuelve los vértices del grafo '''
        return list(self.__grafo_dic.keys())

    def enlaces(self):
        ''' Devuelve los enlaces del grafo '''
        return self.__generar_enlaces()
    
    def vecinos(self, vertice):
        ''' Devuelve los vecinos de ese vértice '''
        return self.__grafo_dic[vertice]

    def anadir_vertice(self, vertice):
        ''' 
            Si el vértice "vertice" no está en 
            self.__grafo_dic, una clave "vertice" con una
            lista vacía como valor es añadida al diccionario. 
            En otro caso, no debe hacerse nada. 
        '''
        if vertice not in self.__grafo_dic:
            self.__grafo_dic[vertice] = []

    def anadir_enlace(self, vertice1, vertice2, longitud):
        ''' 
            Aquí como el grafo es no digirido, el orden de 
            los vértices no es relevante.
            Se crea un enlace entre los vértices de longitud 
            "longitud", añadiendo dichos vértices si es
            necesario y posible.
        '''
        self.anadir_vertice(vertice1)
        self.anadir_vertice(vertice2)
        self.__grafo_dic[vertice1].append((vertice2, longitud))
        self.__grafo_dic[vertice2].append((vertice1, longitud))

    def __generar_enlaces(self):
        ''' 
            Un método estático que genera los enlaces del
            grafo. Los enlaces son representados como 
            diccionarios con "e" (enlace) y "l" (longitud)
        '''
        enlaces = []
        for vertice in self.__grafo_dic:
            for vecino, longitud in self.__grafo_dic[vertice]:
                if {"e" : {vecino, vertice},
                    "l" : longitud} not in enlaces:
                    enlaces.append({"e" : {vertice, vecino},
                                    "l" : longitud})
        return enlaces

    def __str__(self):
        res = "vértices: "
        for k in self.__grafo_dic:
            res += str(k) + " "
        res += "\nenlaces: "
        for enlace in self.__generar_enlaces():
            res += str(enlace) + " "
        return res

In [4]:
import numpy as np

'''
 Función que ordena la lista de enlaces por longitud ascendente
 
 Entrada:
     enlaces: list -> lista de enlaces
 Salida:
     list -> Lista de enlaces ordenada
'''
def ordenar_lista_enlaces(enlaces: list) -> list:
    return sorted(enlaces, key=lambda x: x["l"])

In [5]:
'''
 Función que selecciona el enlace más corto,
 lo extrae de la lista de enlaces y lo devuelve
 
 Entrada:
     enlaces: list -> lista de enlaces ya ordenada
 Salida:
     dict -> enlace (como diccionario) de la primera
             posición de la lista
'''
def seleccionar_kruskal(enlaces: list) -> dict:
    return enlaces.pop(0)

In [6]:
'''
 Función que busca la componente que contiene al 
 vértice v y la devuelve
 
 Entrada:
     bosque: list -> bosque que contiene todas las
                     componentes
     v: str -> vértice a buscar
 Salida:
     set -> componente (en forma de conjunto) donde
            se encuentra el vértice v
'''
def encontrar(bosque: list, v: str) -> set:
    for comp in bosque:
        if v in comp:
            return comp

In [7]:
'''
 Función que combina dos componentes en un bosque
 
 Entrada:
     bosque: list -> bosque que contiene todas las
                     componentes
     comp1: set   -> primera componente
     comp2: set   -> segunda componente
 Salida:
     Nada
'''
def combinar(bosque: list, comp1: set, comp2: set):
    bosque.remove(comp1)
    bosque.remove(comp2)
    bosque.append(comp1.union(comp2))

In [8]:
'''
 Algoritmo de Kruskal para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido
 
 Entrada:
     grafo: lisgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
 Salida:
     (list, float) -> tupla formada por la lista de enlaces del 
                      árbol recubridor mínimo y su longitud total
'''
def kruskal(grafo: lisgraph) -> (list, float):
    longitud_total = 0                # Longitud total del MST
    T = []                            # Lista de enlaces del MST
    
    # Inicializamos un conjunto por cada vértice del grafo
    bosque = [{vertice} for vertice in grafo.vertices()] 
    n = len(bosque)                   # Número de vértices del grafo
    
    # Generamos la lista ordenada de enlaces
    enlaces_ordenados = ordenar_lista_enlaces(grafo.enlaces())
    
    while len(T) < n-1:
        e_optimo = seleccionar_kruskal(enlaces_ordenados)
        (u, v) = e_optimo["e"]
        ucomp, vcomp = encontrar(bosque, u), encontrar(bosque, v)
        if ucomp != vcomp:
            combinar(bosque, ucomp, vcomp)
            T.append(e_optimo)
            longitud_total += e_optimo["l"]
    
    return T, longitud_total

Probamos el algoritmo con el ejemplo de antes

In [9]:
grafo_tfg_lis = lisgraph(g)

kruskal(grafo_tfg_lis)

([{'e': {'3', '4'}, 'l': 1},
  {'e': {'4', '6'}, 'l': 1},
  {'e': {'3', '5'}, 'l': 2},
  {'e': {'1', '2'}, 'l': 3},
  {'e': {'1', '3'}, 'l': 4}],
 11)

<p align="center">
  <img src="imgs/mst_tfg_grafos.png">
</p>

Ahora vamos a implementar la versión donde usamos **min heaps** para representar los enlaces, teniendo el más corto en la raíz. De este modo, el *seleccionar()* será la propia raíz.

In [10]:
import heapq as hq

'''
 Función que crea un min heap según la longitud 
 de los enlaces
 
 Entrada:
     enlaces: list -> lista de enlaces
 Salida:
     list -> heap (como una lista) de enlaces 
             (como tuplas (longitud, enlace))
'''
def crear_min_heap(enlaces: list) -> list:
    heap = []
    for enlace in enlaces:
        hq.heappush(heap, (enlace["l"], enlace["e"]))
    return heap

In [11]:
'''
 Función que selecciona el enlace más corto,
 lo extrae del heap y lo devuelve
 
 Entrada:
     heap: list -> lista de enlaces con estructura de min heap
 Salida:
     dict -> enlace de la raíz del heap, en forma de diccionario
'''
def seleccionar_kruskal_heaps(heap: list) -> dict:
    longitud, enlace = hq.heappop(heap)
    return { "e" : enlace, "l" : longitud }

In [12]:
'''
 Algoritmo de Kruskal para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido, usando un heap para representar
 la lista de enlaces
 
 Entrada:
     grafo: lisgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
 Salida:
     (list, float) -> tupla formada por la lista de enlaces del 
                      árbol recubridor mínimo y su longitud total
'''
def kruskal_heaps(grafo: lisgraph) -> (list, float):
    longitud_total = 0                # Longitud total del MST
    T = []                            # Lista de enlaces del MST
    
    # Inicializamos un conjunto por cada vértice del grafo
    bosque = [{vertice} for vertice in grafo.vertices()] 
    n = len(bosque)                   # Número de vértices del grafo
    
    # Generamos el min heap de enlaces
    heap = crear_min_heap(grafo.enlaces())

    while len(T) < n-1:
        e_optimo = seleccionar_kruskal_heaps(heap)
        (u, v) = e_optimo["e"]
        ucomp, vcomp = encontrar(bosque, u), encontrar(bosque, v)
        if ucomp != vcomp:
            combinar(bosque, ucomp, vcomp)
            T.append(e_optimo)
            longitud_total += e_optimo["l"]

    return T, longitud_total

Volvemos a probar el algoritmo con el ejemplo anterior

In [13]:
kruskal_heaps(grafo_tfg_lis)

([{'e': {'3', '4'}, 'l': 1},
  {'e': {'4', '6'}, 'l': 1},
  {'e': {'3', '5'}, 'l': 2},
  {'e': {'1', '2'}, 'l': 3},
  {'e': {'1', '5'}, 'l': 4}],
 11)

Vemos que el árbol recubridor mínimo cambia en un enlace, pero es igual de óptimo que el otro ya que mantiene la longitud total. Vamos a hacer una pequeña comparación de tiempos entre ambos modos de implementar el algoritmo

In [14]:
import timeit as t

# Encapsulamos los algoritmos en unas funciones sin argumentos para usar luego el módulo timeit
def test_kruskal():
    kruskal(grafo_tfg_lis)
    
def test_kruskal_heaps():
    kruskal_heaps(grafo_tfg_lis)

Vamos a realizar por ahora solo 10000 pruebas donde ejecutamos 100 veces cada algoritmo, para hacer una estadística rápida (también mediremos el tiempo de ejecución completo de la batería de pruebas con el *magic command* **%time**, para una primera comparación)

In [15]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 30 segundos
%time tiempos_kruskal = t.repeat(test_kruskal, repeat=10000, number=100)

CPU times: user 30.7 s, sys: 62 ms, total: 30.8 s
Wall time: 32.6 s


In [16]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 35 segundos
%time tiempos_kruskal_heaps = t.repeat(test_kruskal_heaps, repeat=10000, number=100)

CPU times: user 33.4 s, sys: 31 ms, total: 33.4 s
Wall time: 33.6 s


Imprimamos solo los 10 primeros resultados de cada test, por mostrar el orden de los tiempos:

In [17]:
show("Tiempos Kruskal", tiempos_kruskal[:10])
show("Tiempos Kruskal heaps", tiempos_kruskal_heaps[:10])

'Tiempos Kruskal' [0.0029235000256448984,
 0.0028852999676018953,
 0.002914800017606467,
 0.003541500016581267,
 0.003132999991066754,
 0.0028870999813079834,
 0.002879599982406944,
 0.002912700001616031,
 0.0036557999555952847,
 0.003097900014836341]

'Tiempos Kruskal heaps' [0.006076799996662885,
 0.0033681000350043178,
 0.004047000024002045,
 0.0038690000073984265,
 0.003889799991156906,
 0.004147099971305579,
 0.004475900030229241,
 0.0032955999486148357,
 0.003734799975063652,
 0.003285299986600876]

In [18]:
heaps = 0
for i in range(len(tiempos_kruskal)):
    if tiempos_kruskal[i] > tiempos_kruskal_heaps[i]:
        heaps += 1

show(f"En la prueba hay {heaps} veces que es mejor usar heaps y {i-heaps+1} que no")

'En la prueba hay 1909 veces que es mejor usar heaps y 8091 que no'

### En conclusión, parece que en el ejemplo del TFG solo alrededor del 15-20% de las veces es mejor usar heaps. Sin embargo, estas pruebas deberían realizarse con grafos más aleatorios y mayor número de ejecuciones

# Algoritmo de Prim / Prim algorithm

Vamos a implementar el algoritmo de Prim

<p align="center">
  <img src="imgs/psc_prim_tfg.png">
</p>

La manera de representar el grafo va a ser basada en la clase ***adjgraph*** mencionada en el libro.

La clase ***adjgraph*** contiene:
* Una lista de vértices, cada uno representado por su etiqueta. Asumimos que son números enteros y ordenados del 1 en adelante.
* Una matriz de adyacencias (i.e., una lista de listas) cuadrada y simétrica que contiene en su casilla [i,j] la longitud del enlace que conecta dichos vértices, o infinito (_math.inf_) en caso de no estar conectados

In [58]:
from math import inf

# Grafo de ejemplo del TFG, representado como adjgraph
vertices = ["1","2","3","4","5","6"]
matriz = [[inf, 3, 4, inf, 4, inf],
          [3, inf, inf, 5, inf, inf],
          [4, inf, inf, 1, 2, 4],
          [inf, 5, 1, inf, 3, 1],
          [4, inf, 2, 3, inf, inf],
          [inf, inf, 4, 1, inf, inf]]

In [20]:
'''
    Clase que representa un grafo ponderado NO DIRIGIDO
    de n vértices como:
    - Una lista con las etiquetas de los vértices (ordenados
      de 1 a n)
    - Una matriz de adyacencias simétrica con la longitud de cada
      enlace e infinito en los enlaces que no existan
    Se implementan luego algunos métodos útiles
'''

class adjgraph(object):
    
    def __init__(self, vertices=None, matriz=None):
        '''
            Inicializa un objeto adjgraph.
            Si no se pasan la matriz o los vértices,
            se usarán unos vacíos.
        '''
        if vertices == None:
            vertices = []
        if matriz == None:
            matriz = []
        self.__vertices = vertices
        self.__matriz = matriz

    def vertices(self):
        ''' Devuelve los vértices del grafo '''
        return self.__vertices
    
    def matriz(self):
        ''' Devuelve la matriz de adyacencias del grafo '''
        return self.__matriz

    def enlaces(self, vertice=None):
        ''' 
            Devuelve los enlaces del grafo. 
            Si se pasa un vértice como argumento,
            se devuelven los enlaces que contienen
            a dicho vértice. Si no, se devuelven
            todos los enlaces del grafo
        '''
        return self.__generar_enlaces(vertice)

    def anadir_vertice(self, vertice):
        ''' 
            Si el vértice "vertice" no está en 
            self.__vertices, se añade a la lista al
            final (debe ser el vértice n+1 tal y como
            hemos definido los vértices en este grafo)
            se añade en la matriz de enlaces con inf,
            y se devuelve True
            En otro caso, devuelve False. 
        '''
        if vertice not in self.__vertices:
            num_v = len(self.__vertices)
            if int(vertice) == num_v+1:
                self.__vertices.append(vertice)
                for lista in self.__matriz:
                    lista.append(inf)
                self.__matriz.append([inf]*(num_v+1))
                return True
        return False

    def anadir_enlace(self, vertice1, vertice2, longitud):
        ''' 
            Aquí como el grafo es no digirido, el orden de 
            los vértices solo es relevante a la hora de
            insertar los nuevos vértices en orden.
            Se crea un enlace entre los vértices de longitud 
            "longitud", añadiendo dichos vértices si es
            necesario y posible.
        '''
        indice1, indice2 = int(vertice1), int(vertice2)
        if vertice1 in self.__vertices and vertice2 in self.__vertices:
            i, j = indice1-1, indice2-1 
            if self.__matriz[i][j] == inf:
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
        elif vertice1 in self.__vertices:
            if self.anadir_vertice(vertice2):
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
        else:
            if abs(indice1-indice2) == 1:
                if self.anadir_vertice(vertice1):
                    self.anadir_vertice(vertice2)
                elif self.anadir_vertice(vertice2):
                    self.anadir_vertice(vertice1)
                else:
                    return
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
                    

    def __generar_enlaces(self, vertice=None):
        ''' 
            Un método estático que genera los enlaces del
            grafo. Si se pasa un vértice como argumento,
            se generan los enlaces que contienen
            a dicho vértice. Si no, se generan todos.
            Los enlaces son representados como 
            diccionarios con "e" (enlace) y "l" (longitud)
        '''
        enlaces = []
        n = len(self.__vertices)
        if vertice is None:
            for i in range(n):
                for j in range(i, n):
                    enlace = self.__crear_enlace(i, j)
                    if enlace is not None:
                        enlaces.append(enlace)
                    
        else:
            j = int(vertice - 1)
            for i in range(n):
                enlace = self.__crear_enlace(i, j)
                if enlace is not None:
                    enlaces.append(enlace)
                        
        return enlaces
    
    def __crear_enlace(self, fila, columna):
        ''' 
            Un método estático que crea un enlace
            accediendo a la matriz de adyacencias
            por una fila y columna determinadas
        '''
        if self.__matriz[fila][columna] != inf:
            return {"e" : {fila + 1, columna + 1},
                    "l" : self.__matriz[fila][columna]}
        else:
            return None

    def __str__(self):
        res = "vértices: "
        for k in self.__vertices:
            res += k + " "
        res += "\nenlaces: "
        for enlace in self.__generar_enlaces():
            res += str(enlace) + " "
        return res

In [21]:
'''
 Algoritmo de Prim para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
 Salida:
     (list, float) -> tupla formada por la lista de enlaces del 
                      árbol recubridor mínimo y su longitud total
'''
def prim(grafo: adjgraph) -> (set, float):
    longitud_total = 0                 # Longitud total del MST
    T = []                             # Lista de enlaces del MST
    primero = int(grafo.vertices()[0]) # Asumimos comenzar por el primer vértice
    n = len(grafo.vertices())          # Número de vértices del grafo
    matriz = grafo.matriz()            # Matriz de adyacencias del grafo
    
    # Inicializamos nuestros arrays según el vértice por el que empezamos
    mas_cercano = [primero] * n                 
    dist_min = [-1]
    for i in range(1, n):
        dist_min.append(matriz[i][primero - 1])

    for _ in range(n-1):
        min = inf
        for j in range(1, n):
            if 0 <= dist_min[j] and dist_min[j] < min:
                min = dist_min[j]
                k = j
        T.append({ "e" : {mas_cercano[k], k + 1},
                   "l" : min })
        longitud_total += min
        dist_min[k] = -1
        for j in range(1, n):
            if matriz[j][k] < dist_min[j]:
                dist_min[j] = matriz[j][k]
                mas_cercano[j] = k + 1
    
    return T, longitud_total

Probamos el algoritmo con el ejemplo anterior

In [22]:
grafo_tfg_adj = adjgraph(vertices, matriz)

prim(grafo_tfg_adj)

([{'e': {1, 2}, 'l': 3},
  {'e': {1, 3}, 'l': 4},
  {'e': {3, 4}, 'l': 1},
  {'e': {4, 6}, 'l': 1},
  {'e': {3, 5}, 'l': 2}],
 11)

Podemos observar que conseguimos el mismo MST que conseguimos con Kruskal la primera vez.

Ahora vamos a implementar la versión donde usamos **min heaps** para representar los enlaces, teniendo el más corto en la raíz. De este modo, el *seleccionar()* será la propia raíz. De todos modos, esta vez no vamos a implementar otro *seleccionar()* y usaremos el propio *heapq.heappop()*

Para este caso parece óptimo usar la clase *lisgraph* de nuevo en lugar de seguir con *adjgraph*, ya que la idea no es crear el heap desde el comienzo sino irlo creando paso a paso añadiendo los enlaces de los vecinos del nodo a considerar

In [23]:
'''
 Algoritmo de Prim para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido, usando un heap para representar
 la lista de enlaces sin explorar
 
 Entrada:
     grafo: lisgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
 Salida:
     (list, float) -> tupla formada por la lista de enlaces del 
                      árbol recubridor mínimo y su longitud total
'''

def prim_heaps_lis(grafo: lisgraph) -> (list, float):
    longitud_total = 0                # Longitud total del MST
    explorados = set()                # Conjunto de vértices del MST
    primero = grafo.vertices()[0]     # Asumimos comenzar por el primer vértice
    T = []                            # Lista de enlaces del MST
    
    # Enlaces no explorados ordenados por longitud (será nuestro heap)
    no_explorados = [(0, primero)]   
    
    while no_explorados:
        longitud, ganador = hq.heappop(no_explorados)
        if explorados:
            (vertice1, vertice2) = tuple(ganador)
            if vertice1 not in explorados:
                nuevo_vertice = vertice1
                T.append({ "e" : ganador, 
                           "l" : longitud })
            elif vertice2 not in explorados:
                nuevo_vertice = vertice2
                T.append({ "e" : ganador, 
                           "l" : longitud })
            else:
                continue
        else: 
            nuevo_vertice = ganador
        explorados.add(nuevo_vertice)
        longitud_total += longitud
        vecinos = grafo.vecinos(nuevo_vertice)
        for vecino, longitud in vecinos:
            if vecino not in explorados:
                hq.heappush(no_explorados, (longitud, {nuevo_vertice, vecino}))
    return T, longitud_total

Volvemos a probar el algoritmo con el mismo ejemplo

In [24]:
prim_heaps_lis(grafo_tfg_lis)

([{'e': {'1', '2'}, 'l': 3},
  {'e': {'1', '3'}, 'l': 4},
  {'e': {'3', '4'}, 'l': 1},
  {'e': {'4', '6'}, 'l': 1},
  {'e': {'3', '5'}, 'l': 2}],
 11)

Obtenemos el mismo MST.

Vamos a hacer una pequeña comparación de tiempos entre ambos modos de implementar el algoritmo, tal y como hicimos antes

In [25]:
def test_prim():
    prim(grafo_tfg_adj)
    
def test_prim_heaps_lis():
    prim_heaps_lis(grafo_tfg_lis)

Repitamos el proceso de pruebas anterior:

In [26]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 25 segundos
%time tiempos_prim = t.repeat(test_prim, repeat=10000, number=100)

CPU times: user 23.1 s, sys: 0 ns, total: 23.1 s
Wall time: 23.5 s


In [27]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 15 segundos
%time tiempos_prim_heaps_lis = t.repeat(test_prim_heaps_lis, repeat=10000, number=100)

CPU times: user 12.1 s, sys: 0 ns, total: 12.1 s
Wall time: 12.1 s


In [28]:
show("Tiempos Prim", tiempos_prim[:10])
show("Tiempos Prim heaps lisgraph", tiempos_prim_heaps_lis[:10])

'Tiempos Prim' [0.004213999956846237,
 0.0038713000249117613,
 0.00420619995566085,
 0.002959400007966906,
 0.00282100000185892,
 0.0025659999810159206,
 0.002564000023994595,
 0.0025633000186644495,
 0.002640100021380931,
 0.002338999998755753]

'Tiempos Prim heaps lisgraph' [0.0011963999713771045,
 0.0011727000237442553,
 0.0011706000077538192,
 0.0011700000031851232,
 0.0011696000001393259,
 0.0011678999871946871,
 0.0011680999887175858,
 0.0011675999849103391,
 0.0011700000031851232,
 0.0014205999905243516]

In [29]:
heaps = 0
for i in range(len(tiempos_prim)):
    if tiempos_prim[i] > tiempos_prim_heaps_lis[i]:
        heaps += 1

show(f"En la prueba hay {heaps} veces que es mejor usar heaps y {i-heaps+1} que no")

'En la prueba hay 9991 veces que es mejor usar heaps y 9 que no'

### En conclusión, parece que en el ejemplo del TFG casi siempre es mejor usar heaps (aproximadamente en un 99% de los casos).

He tenido que volver a reutilizar la clase *lisgraph* para los grafos en la implementación de Prim con heaps, pero tengo curiosidad por ver cómo se implementaría reusando *adjgraph* y comparar tiempos de nuevo. Mi intuición me dice que va a ser más lento ya que la idea de ir creando el heap poco a poco se aprovechaba mucho de tener la lista de vecinos

In [30]:
'''
 Algoritmo de Prim para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido, usando un heap para representar
 la lista de enlaces sin explorar
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
 Salida:
     (list, float) -> tupla formada por la lista de enlaces del 
                      árbol recubridor mínimo y su longitud total
'''

def prim_heaps_adj(grafo: adjgraph) -> (list, float):
    longitud_total = 0                # Longitud total del MST
    explorados = set()                # Conjunto de vértices del MST
    primero = grafo.vertices()[0]     # Asumimos comenzar por el primer vértice
    T = []                            # Lista de enlaces del MST
    matriz = grafo.matriz()           # Matriz de adyacencias del grafo
    
    # Enlaces no explorados ordenados por longitud (será nuestro heap)
    no_explorados = [(0, primero)]   
    
    while no_explorados:
        longitud, ganador = hq.heappop(no_explorados)
        if explorados:
            (vertice1, vertice2) = tuple(ganador)
            if vertice1 not in explorados:
                nuevo_vertice = vertice1
                T.append({ "e" : ganador, 
                           "l" : longitud })
            elif vertice2 not in explorados:
                nuevo_vertice = vertice2
                T.append({ "e" : ganador, 
                           "l" : longitud })
            else:
                continue
        else: 
            nuevo_vertice = ganador
        explorados.add(nuevo_vertice)
        longitud_total += longitud
        vecino = primero
        for longitud in matriz[int(nuevo_vertice)-1]:
            if longitud < inf and vecino not in explorados:
                hq.heappush(no_explorados, (longitud, {nuevo_vertice, vecino}))
            vecino = str(int(vecino) + 1)
    return T, longitud_total

In [31]:
prim_heaps_adj(grafo_tfg_adj)

([{'e': {'1', '2'}, 'l': 3},
  {'e': {'1', '3'}, 'l': 4},
  {'e': {'3', '4'}, 'l': 1},
  {'e': {'4', '6'}, 'l': 1},
  {'e': {'3', '5'}, 'l': 2}],
 11)

El funcionamiento sigue siendo el adecuado, veamos ahora la eficiencia

In [32]:
def test_prim_heaps_adj():
    prim_heaps_adj(grafo_tfg_adj)

Mismas pruebas

In [33]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda es de 1 minuto
%time tiempos_prim_heaps_adj = t.repeat(test_prim_heaps_adj, repeat=10000, number=100)

CPU times: user 1min 6s, sys: 422 ms, total: 1min 6s
Wall time: 1min 6s


In [34]:
show("Tiempos Prim heaps adjgraph", tiempos_prim_heaps_adj[:10])

'Tiempos Prim heaps adjgraph' [0.005472699995152652,
 0.005302099976688623,
 0.006413700000848621,
 0.005542300001252443,
 0.006572200043592602,
 0.0056856999872252345,
 0.006382399995345622,
 0.005789100017864257,
 0.005483800021465868,
 0.0053186999866738915]

In [35]:
lis = 0
for i in range(len(tiempos_prim_heaps_adj)):
    if tiempos_prim_heaps_adj[i] > tiempos_prim_heaps_lis[i]:
        lis += 1

show(f"En la prueba hay {lis} veces que es mejor usar lisgraph y {i-lis+1} que no")

'En la prueba hay 9998 veces que es mejor usar lisgraph y 2 que no'

### En conclusión, parece que en el ejemplo del TFG casi siempre es mejor usar la clase *lisgraph* (aproximadamente en un 99% de los casos). Incluso, parece que usar heaps con *adjgraph* resulta bastante menos eficiente que no usar heaps.

# Algoritmo de Dijkstra / Dijkstra algorithm

Vamos a implementar el algoritmo de Dijkstra, que trata de resolver el problema de dado un grafo conexo ponderado dirigido, encontrar los caminos más cortos desde un vértice a cada uno de los otros. En la memoria no mencionamos este problema, pero lo hemos sacado del libro *Fundamentals of algorithmics. Chapter 6: Greedy algorithms (Brassard, G.; Bratley, P.; 1996)*.

<p align="center">
  <img src="imgs/psc_dijkstra_tfg.png">
</p>

Para determinar no solo la longitud de los caminos más cortos, pero también por dónde pasan, habría que añadir un segundo *array P[2..n]*, donde *P[v]* contiene la etiqueta del vértice que precede a _v_ en el camino más corto. Para encontrar el camino completo, hay que seguir los punteros de *P* hacia detrás desde el destino hasta el origen. Estas serían las modificaciones necesarias del pseudocódigo anterior:

<p align="center">
  <img src="imgs/psc_dijkstra_tfg_2.png">
</p>

Parece que el pseudocódigo muestra el grafo como *adjgraph* al existir esa matriz de adyacencias. Comencemos implementándolo así, aunque más adelante el libro recomienda una versión usando *lisgraph*.

De todos modos, ahora pasamos de grafos **no dirigidos** a grafos **dirigidos**, por lo que tendremos que reimplementar nuestras clases con algunas pequeñas modificaciones. También a la hora de implementar enlaces, ahora no van a ser un conjunto sino una **tupla**

In [36]:
'''
    Clase que representa un grafo ponderado DIRIGIDO 
    de n vértices como:
    - Una lista con las etiquetas de los vértices (ordenados
      de 1 a n)
    - Una matriz de adyacencias (no necesariamente simétrica) 
      con la longitud de cada enlace e infinito en los enlaces 
      que no existan
    Se implementan luego algunos métodos útiles
'''

class adjgraph_dir(object):
    
    def __init__(self, vertices=None, matriz=None):
        '''
            Inicializa un objeto adjgraph_dir.
            Si no se pasan la matriz o los vértices,
            se usarán unos vacíos.
        '''
        if vertices == None:
            vertices = []
        if matriz == None:
            matriz = []
        self.__vertices = vertices
        self.__matriz = matriz

    def vertices(self):
        ''' Devuelve los vértices del grafo '''
        return self.__vertices
    
    def matriz(self):
        ''' Devuelve la matriz de adyacencias del grafo '''
        return self.__matriz

    def enlaces(self, vertice=None):
        ''' 
            Devuelve los enlaces del grafo. 
            Si se pasa un vértice como argumento,
            se devuelven los enlaces que contienen
            a dicho vértice. Si no, se devuelven
            todos los enlaces del grafo
        '''
        return self.__generar_enlaces(vertice)

    def anadir_vertice(self, vertice):
        ''' 
            Si el vértice "vertice" no está en 
            self.__vertices, se añade a la lista al
            final (debe ser el vértice n+1 tal y como
            hemos definido los vértices en este grafo)
            se añade en la matriz de enlaces con inf,
            y se devuelve True
            En otro caso, devuelve False. 
        '''
        if vertice not in self.__vertices:
            num_v = len(self.__vertices)
            if int(vertice) == num_v+1:
                self.__vertices.append(vertice)
                for lista in self.__matriz:
                    lista.append(inf)
                self.__matriz.append([inf]*(num_v+1))
                return True
        return False

    def anadir_enlace(self, origen, destino, longitud):
        ''' 
            Aquí como el grafo es digirido, el orden de 
            los vértices es importante. Se crea un enlace
            de "origen" a "destino" de longitud "longitud",
            añadiendo los vértices si es necesario y posible.
        '''
        indice1, indice2 = int(origen), int(destino)
        if origen in self.__vertices and destino in self.__vertices:
            i, j = indice1-1, indice2-1 
            if self.__matriz[i][j] == inf:
                self.__matriz[i][j] = longitud
        elif origen in self.__vertices:
            if self.anadir_vertice(destino):
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
        elif destino in self.__vertices:
            if self.anadir_vertice(origen):
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
        else:
            if abs(indice1-indice2) == 1:
                if self.anadir_vertice(origen):
                    self.anadir_vertice(destino)
                elif self.anadir_vertice(destino):
                    self.anadir_vertice(origen)
                else:
                    return
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud

    def __generar_enlaces(self, vertice=None):
        ''' 
            Un método estático que genera los enlaces del
            grafo. Si se pasa un vértice como argumento,
            se generan los enlaces que contienen
            a dicho vértice. Si no, se generan todos.
            Los enlaces son representados como 
            diccionarios con "e" (enlace) y "l" (longitud)
        '''
        enlaces = []
        n = len(self.__vertices)
        if vertice is None:
            for i in range(n):
                for j in range(n):
                    enlace = self.__crear_enlace(i, j)
                    if enlace is not None:
                        enlaces.append(enlace)
        else:
            j = int(vertice - 1)
            for i in range(n):
                enlace = self.__crear_enlace(i, j)
                if enlace is not None:
                    enlaces.append(enlace)
                enlace = self.__crear_enlace(j, i)
                if enlace is not None:
                    enlaces.append(enlace)
                
        return enlaces
    
    def __crear_enlace(self, fila, columna):
        ''' 
            Un método estático que crea un enlace
            accediendo a la matriz de adyacencias
            por una fila y columna determinadas
        '''
        if self.__matriz[fila][columna] != inf:
            return {"e" : (fila + 1, columna + 1),
                    "l" : self.__matriz[fila][columna]}
        else:
            return None

    def __str__(self):
        res = "vértices: "
        for k in self.__vertices:
            res += k + " "
        res += "\nenlaces: "
        for enlace in self.__generar_enlaces():
            res += str(enlace) + " "
        return res

In [37]:
# Ejemplo de enlace dirigido como diccionario
a = { "e" : ("1","2"),
      "l" : 1 }

In [38]:
# Grafo de ejemplo del libro, de la figura 6.3, representado como adjgraph_dir
vertices = ["1","2","3","4","5"]
matriz = [[inf, 50, 30, 100, 10],
          [inf, inf, inf, inf, inf],
          [inf, 5, inf, inf, inf],
          [inf, 20, 50, inf, inf],
          [inf, inf, inf, 10, inf]]

In [39]:
'''
 Función que selecciona el vértice a menos distancia,
 lo extrae del conjunto de no explorados y lo devuelve
 
 Entrada:
     no_explorados: set -> conjunto de vértices no explorados
     distancias: list   -> lista de distancias al origen
 Salida:
     (float, str) -> tupla (distancia, etiqueta) del vértice
                     a menor distancia
'''
def seleccionar_dijkstra(no_explorados: set, distancias: list) -> (float, str):
    min = inf
    for vertice in no_explorados:
        indice = int(vertice) - 2
        if distancias[indice] < min:
            min = distancias[indice]
            indice_ganador = indice
    ganador = str(indice_ganador + 2)
    no_explorados.remove(ganador)
    return min, ganador

In [40]:
'''
 Algoritmo de Dijkstra para encontrar el camino más corto entre
 un vértice origen y cada uno del resto de los vértices
 
 Entrada:
     grafo: adjgraph_dir -> grafo donde buscamos los caminos 
                            más cortos
 Salida:
     (list, list) -> tupla formada por la lista de longitudes de 
                     los caminos más cortos y una lista de punteros
                     que indica el vértice anterior a seguir en
                     cada camino
'''
def dijkstra_adj(grafo: adjgraph_dir) -> (list, list):
    D = []                                     # Lista de distancias al origen
    no_explorados = grafo.vertices().copy()    # Lista de vértices no explorados, ordenados
    n = len(no_explorados)                     # Número de vértices del grafo
    primero =  no_explorados.pop(0)            # Asumimos comenzar por el primer vértice
    no_explorados = set(no_explorados)         # Hacemos que C sea un conjunto
    P = [primero] * (n - 1)                    # Lista de punteros que indica cada camino
    matriz = grafo.matriz()                    # Matriz de adyacencias del grafo
    
    # Inicializamos nuestro array según el vértice por el que empezamos
    for i in range(1, n):
        D.append(matriz[int(primero) - 1][i])

    for _ in range(n-2):
        dist_min, v = seleccionar_dijkstra(no_explorados, D)
        for w in no_explorados:
            indice_v, indice_w = int(v)-2, int(w)-2
            suma = dist_min + matriz[indice_v + 1][indice_w + 1]
            if D[indice_w] > suma:
                D[indice_w] = suma
                P[indice_w] = v
    
    return D, P

Probamos el algoritmo con el ejemplo del libro

In [41]:
grafo_dir_adj = adjgraph_dir(vertices, matriz)

dijkstra_adj(grafo_dir_adj)

([35, 30, 20, 10], ['3', '1', '5', '1'])

¿Podríamos hacer lo mismo con *lisgraph*? Vamos a reimplementar la clase *lisgraph* ahora para grafos dirigidos, tal y como hicimos con *adjgraph*

In [42]:
'''
    Clase que representa un grafo DIRIGIDO como un diccionario con:
    - Las etiquetas de los vértices como claves
    - Una lista con tuplas de (etiqueta, longitud) de los vecinos
      de dicho vértice y con cada longitud de esos enlaces
    Se implementan luego algunos métodos útiles
'''

class lisgraph_dir(object):
    
    def __init__(self, grafo_dic=None):
        '''
            Inicializa un objeto lisgraph_dir.
            Si no se pasa el diccionario,
            se usará uno vacío.
        '''
        if grafo_dic == None:
            grafo_dic = {}
        self.__grafo_dic = grafo_dic

    def vertices(self):
        ''' Devuelve los vértices del grafo '''
        return list(self.__grafo_dic.keys())

    def enlaces(self):
        ''' Devuelve los enlaces del grafo '''
        return self.__generar_enlaces()
    
    def vecinos(self, vertice):
        ''' Devuelve los vecinos de ese vértice '''
        return self.__grafo_dic[vertice]

    def anadir_vertice(self, vertice):
        ''' 
            Si el vértice "vertice" no está en 
            self.__grafo_dic, una clave "vertice" con una
            lista vacía como valor es añadida al diccionario. 
            En otro caso, no debe hacerse nada. 
        '''
        if vertice not in self.__grafo_dic:
            self.__grafo_dic[vertice] = []

    def anadir_enlace(self, origen, destino, longitud):
        ''' 
            Aquí como el grafo es digirido, el orden de 
            los vértices es importante. Se crea un enlace
            de "origen" a "destino" de longitud "longitud",
            añadiendo los vértices si es necesario y posible.
        '''
        self.anadir_vertice(destino)
        if origen in self.__grafo_dic:
            self.__grafo_dic[origen].append((destino, longitud))
        else:
            self.__grafo_dic[origen] = [(destino, longitud)]

    def __generar_enlaces(self):
        ''' 
            Un método estático que genera los enlaces del
            grafo. Los enlaces son representados como 
            diccionarios con "e" (enlace) y "l" (longitud)
        '''
        enlaces = []
        for vertice in self.__grafo_dic:
            for vecino, longitud in self.__grafo_dic[vertice]:
                if {"e" : (vecino, vertice),
                    "l" : longitud} not in enlaces:
                    enlaces.append({"e" : (vertice, vecino),
                                    "l" : longitud})
        return enlaces

    def __str__(self):
        res = "vértices: "
        for k in self.__grafo_dic:
            res += str(k) + " "
        res += "\nenlaces: "
        for enlace in self.__generar_enlaces():
            res += str(enlace) + " "
        return res

In [43]:
# Grafo de ejemplo del libro, de la figura 6.3, representado como lisgraph_dir
g = { "1" : [("2", 50), ("3", 30), ("4", 100), ("5", 10)],
      "2" : [],
      "3" : [("2", 5)],
      "4" : [("2", 20), ("3", 50)],
      "5" : [("4", 10)] }

In [44]:
'''
 Algoritmo de Dijkstra para encontrar el camino más corto entre
 un vértice origen y cada uno del resto de los vértices
 
 Entrada:
     grafo: lisgraph_dir -> grafo donde buscamos los caminos 
                            más cortos
 Salida:
     (list, list) -> tupla formada por la lista de longitudes de 
                     los caminos más cortos y una lista de punteros
                     que indica el vértice anterior a seguir en
                     cada camino
'''
def dijkstra_lis(grafo: lisgraph_dir) -> (list, list):
    no_explorados = grafo.vertices().copy()    # Lista de vértices no explorados, ordenados
    n = len(no_explorados)                     # Número de vértices del grafo
    primero =  no_explorados.pop(0)            # Asumimos comenzar por el primer vértice
    no_explorados = set(no_explorados)         # Hacemos que C sea un conjunto
    D = [inf] * (n - 1)                        # Lista de distancias al origen
    P = [primero] * (n - 1)                    # Lista de punteros que indica cada camino
    
    # Inicializamos nuestro array según el vértice por el que empezamos
    for vecino, longitud in grafo.vecinos(primero):
        D[int(vecino) - 2] = longitud

    for _ in range(n-2):
        dist_min, v = seleccionar_dijkstra(no_explorados, D)
        vecinos = grafo.vecinos(v)
        for w, longitud in vecinos:
            indice_w = int(w)-2
            suma = dist_min + longitud
            if D[indice_w] > suma:
                D[indice_w] = suma
                P[indice_w] = v
    
    return D, P

Volvemos a probar el algoritmo con el ejemplo del libro

In [45]:
grafo_dir_lis = lisgraph_dir(g)

dijkstra_lis(grafo_dir_lis)

([35, 30, 20, 10], ['3', '1', '5', '1'])

El funcionamiento sigue siendo el adecuado, veamos ahora la eficiencia

In [46]:
def test_dijkstra_adj():
    dijkstra_adj(grafo_dir_adj)

def test_dijkstra_lis():
    dijkstra_lis(grafo_dir_lis)

Continuemos con el mismo estilo de pruebas

In [47]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 30 segundos
%time tiempos_dijkstra_adj = t.repeat(test_dijkstra_adj, repeat=10000, number=100)

CPU times: user 28.4 s, sys: 0 ns, total: 28.4 s
Wall time: 28.5 s


In [59]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 20 segundos
%time tiempos_dijkstra_lis = t.repeat(test_dijkstra_lis, repeat=10000, number=100)

CPU times: user 19.2 s, sys: 0 ns, total: 19.2 s
Wall time: 19.7 s


In [49]:
show("Tiempos Dijkstra adjgraph", tiempos_dijkstra_adj[:10])
show("Tiempos Dijkstra lisgraph", tiempos_dijkstra_lis[:10])

'Tiempos Dijkstra adjgraph' [0.003574000031221658,
 0.0027860000263899565,
 0.0028637999785132706,
 0.003205199958756566,
 0.0037104000221006572,
 0.0029132000054232776,
 0.0032469999860040843,
 0.002803899988066405,
 0.003132699988782406,
 0.0027803999837487936]

'Tiempos Dijkstra lisgraph' [0.0019443000201135874,
 0.0019197000074200332,
 0.0018606000230647624,
 0.0018551999819464982,
 0.0018515000119805336,
 0.0018690999713726342,
 0.0024741999804973602,
 0.0020540999830700457,
 0.0019987999694421887,
 0.0021603000350296497]

In [50]:
lis = 0
for i in range(len(tiempos_dijkstra_adj)):
    if tiempos_dijkstra_adj[i] > tiempos_dijkstra_lis[i]:
        lis += 1

show(f"En la prueba hay {lis} veces que es mejor usar lisgraph y {i-lis+1} que no")

'En la prueba hay 9990 veces que es mejor usar lisgraph y 10 que no'

### En conclusión, parece que en el ejemplo del libro casi siempre es mejor usar la clase *lisgraph* (aproximadamente en un 99% de los casos).

Para finalizar la implementación de estos algoritmos, vamos a seguir la recomendación del libro una vez más y usar **min heaps** para representar el conjunto de vértices no explorados, ordenando por distancia al origen. De ese modo podremos hacer el *seleccionar()* en tiempo logarítmico. Mantenemos la utilización de *lisgraph* ya que era más eficiente, al menos para este ejemplo.

In [51]:
'''
 Función que selecciona el vértice a menos distancia,
 lo extrae del heap de no explorados y lo devuelve
 
 Entrada:
     heap: list -> heap de vértices no explorados
 Salida:
     (float, str) -> raíz del heap
'''
def seleccionar_dijkstra_heaps(heap: list) -> (float, str):
    return hq.heappop(heap)

In [52]:
'''
 Algoritmo de Dijkstra para encontrar el camino más corto entre
 un vértice origen y cada uno del resto de los vértices, usando
 un min heap para el conjunto de vértices no explorados
 
 Entrada:
     grafo: lisgraph_dir -> grafo donde buscamos los caminos 
                            más cortos
 Salida:
     (list, list) -> tupla formada por la lista de longitudes de 
                     los caminos más cortos y una lista de punteros
                     que indica el vértice anterior a seguir en
                     cada camino
'''
def dijkstra_lis_heaps(grafo: lisgraph_dir) -> (list, list):
    n = len(grafo.vertices())         # Número de vértices del grafo
    explorados = set()                # Conjunto de vértices del MST
    primero = grafo.vertices()[0]     # Asumimos comenzar por el primer vértice
    D = [inf] * (n - 1)               # Lista de distancias al origen
    P = [primero] * (n - 1)           # Lista de punteros que indica cada camino
    
    # Inicializamos nuestro array según el vértice por el que empezamos
    for vecino, longitud in grafo.vecinos(primero):
        D[int(vecino) - 2] = longitud
    
    no_explorados = [(0, primero)]

    while no_explorados:
        dist_min, v = seleccionar_dijkstra_heaps(no_explorados)
        if explorados:
            if v in explorados:
                continue
        explorados.add(v)
        vecinos = grafo.vecinos(v)
        for w, longitud in vecinos:
            indice_w = int(w)-2
            suma = dist_min + longitud
            if D[indice_w] >= suma and w not in explorados:
                D[indice_w] = suma
                hq.heappush(no_explorados, (suma, w))
                P[indice_w] = v
    
    return D, P

Observamos que hemos vuelto a reutilizar la idea de **ir creando el heap paso a paso con los vecinos no explorados**, pues desde el módulo *heapq* no tendríamos modo de "promocionar" el nodo del heap cuya distancia modificamos.

Probemos por última vez con el ejemplo del libro

In [53]:
dijkstra_lis_heaps(grafo_dir_lis)

([35, 30, 20, 10], ['3', '1', '5', '1'])

Funciona adecuadamente, veamos ahora la eficiencia

In [54]:
def test_dijkstra_lis_heaps():
    dijkstra_lis_heaps(grafo_dir_lis)

Mismas pruebas

In [60]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 20 segundos
%time tiempos_dijkstra_lis_heaps = t.repeat(test_dijkstra_lis_heaps, repeat=10000, number=100)

CPU times: user 18.5 s, sys: 46 ms, total: 18.5 s
Wall time: 22.1 s


In [56]:
show("Tiempos Dijkstra heaps lisgraph", tiempos_dijkstra_lis_heaps[:10])

'Tiempos Dijkstra heaps lisgraph' [0.0018414999940432608,
 0.0018124000052921474,
 0.0018218999612145126,
 0.0018142000189982355,
 0.0018191999988630414,
 0.0020386999822221696,
 0.0022347000194713473,
 0.001954499981366098,
 0.001768199959769845,
 0.0019331999938003719]

In [61]:
heaps = 0
for i in range(len(tiempos_dijkstra_lis)):
    if tiempos_dijkstra_lis[i] > tiempos_dijkstra_lis_heaps[i]:
        heaps += 1

show(f"En la prueba hay {heaps} veces que es mejor usar heaps y {i-heaps+1} que no")

'En la prueba hay 7495 veces que es mejor usar heaps y 2505 que no'

### En conclusión, parece que en el ejemplo del libro es mejor usar heaps en una alta proporción de los casos (por encima del 70%). De todos modos, como en cada caso anterior, estas pruebas deberían realizarse con grafos más aleatorios y mayor número de ejecuciones